In [ ]:
!pip install q kaggle

In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
from google.colab import autoviz
import seaborn as sns

In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d prashant268/chest-xray-covid19-pneumonia

Dataset URL: https://www.kaggle.com/datasets/prashant268/chest-xray-covid19-pneumonia
License(s): unknown
 99% 2.03G/2.06G [00:27<00:00, 95.4MB/s]
100% 2.06G/2.06G [00:27<00:00, 79.3MB/s]


In [ ]:
!unzip -o /content/chest-xray-covid19-pneumonia.zip

Streaming output truncated to the last 5000 lines.
  inflating: Data/train/COVID19/COVID19(228).jpg  
  inflating: Data/train/COVID19/COVID19(229).jpg  
  inflating: Data/train/COVID19/COVID19(23).jpg  
  inflating: Data/train/COVID19/COVID19(230).jpg  
  inflating: Data/train/COVID19/COVID19(231).jpg  
  inflating: Data/train/COVID19/COVID19(232).jpg  
  inflating: Data/train/COVID19/COVID19(233).jpg  
  inflating: Data/train/COVID19/COVID19(234).jpg  
  inflating: Data/train/COVID19/COVID19(235).jpg  
  inflating: Data/train/COVID19/COVID19(236).jpg  
  inflating: Data/train/COVID19/COVID19(237).jpg  
  inflating: Data/train/COVID19/COVID19(238).jpg  
  inflating: Data/train/COVID19/COVID19(239).jpg  
  inflating: Data/train/COVID19/COVID19(24).jpg  
  inflating: Data/train/COVID19/COVID19(240).jpg  
  inflating: Data/train/COVID19/COVID19(241).jpg  
  inflating: Data/train/COVID19/COVID19(242).jpg  
  inflating: Data/train/COVID19/COVID19(243).jpg  
  inflating: Data/train/COVID19/C

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense , Conv2D , MaxPool2D , Dropout , Flatten
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
!mv /content/Data/train/PNEUMONIA /content/Data/unused


In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    rotation_range=30,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(directory="/content/Data/train",target_size=(224,224),batch_size=32,class_mode='binary')

Found 1726 images belonging to 2 classes.


In [ ]:
train_data.class_indices

{'COVID19': 0, 'NORMAL': 1}

In [ ]:
!mv /content/Data/test/PNEUMONIA /content/Data/unused


In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255, validation_split=0.2
)

test_data = test_datagen.flow_from_directory(directory="/content/Data/test",target_size=(224,224),batch_size=32,class_mode='binary')

Found 433 images belonging to 2 classes.


In [ ]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(256,256,3)))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64,activation="relu"))
model.add(Dropout(rate=0.50))
model.add(Dense(units=1,activation="sigmoid"))


model.compile(loss=keras.losses.binary_crossentropy,optimizer="adam",metrics=['acc'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 252, 252, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 126, 126, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 126, 126, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 124, 124, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       7,372,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,503,105 (28.62 MB)

 Trainable params: 7,503,105 (28.62 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the 2D outputs to 1D
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary output

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, steps_per_epoch=8, epochs=10, validation_data=test_data)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 87s 10s/step - accuracy: 0.7322 - loss: 3.1393 - val_accuracy: 0.9400 - val_loss: 0.2836
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 74s 10s/step - accuracy: 0.8994 - loss: 0.3246 - val_accuracy: 0.9515 - val_loss: 0.1446
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 73s 10s/step - accuracy: 0.9048 - loss: 0.2900 - val_accuracy: 0.9353 - val_loss: 0.1521
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 73s 10s/step - accuracy: 0.9397 - loss: 0.2011 - val_accuracy: 0.9630 - val_loss: 0.1024
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 54s 7s/step - accuracy: 0.9216 - loss: 0.1672 - val_accuracy: 0.9792 - val_loss: 0.0669
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 52s 7s/step - accuracy: 0.9245 - loss: 0.1803 - val_accuracy: 0.9723 - val_loss: 0.0733
Epoch 7/10
6/8 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.9576 - loss: 0.1317

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 38s 5s/step - accuracy: 0.9539 - loss: 0.1448 - val_accuracy: 0.9746 - val_loss: 0.0790
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 136s 10s/step - accuracy: 0.9506 - loss: 0.1600 - val_accuracy: 0.9769 - val_loss: 0.0914
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 79s 11s/step - accuracy: 0.9500 - loss: 0.1553 - val_accuracy: 0.9815 - val_loss: 0.0732
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 74s 10s/step - accuracy: 0.9503 - loss: 0.1173 - val_accuracy: 0.9885 - val_loss: 0.0555


In [ ]:
path = "/content/Data/test/COVID19/COVID19(462).jpg"
img = image.load_img(path,target_size=(256,256))

img = image.img_to_array(img)/255
img = np.array([img])
img.shape

(1, 256, 256, 3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save the model to Google Drive
model.save('/content/drive/MyDrive/my_model_Covid19.keras')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a callback to save the best model
checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model_Covid19.keras', monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model with the checkpoint
model.fit(train_data, validation_data=test_data, epochs=10, callbacks=[checkpoint])


Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 256s 5s/step - accuracy: 0.9704 - loss: 0.0827 - val_accuracy: 0.9792 - val_loss: 0.0637
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 244s 4s/step - accuracy: 0.9678 - loss: 0.1040 - val_accuracy: 0.9723 - val_loss: 0.0754
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 269s 4s/step - accuracy: 0.9757 - loss: 0.0832 - val_accuracy: 0.9746 - val_loss: 0.0619
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 244s 4s/step - accuracy: 0.9746 - loss: 0.0829 - val_accuracy: 0.9861 - val_loss: 0.0338
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 259s 5s/step - accuracy: 0.9773 - loss: 0.0612 - val_accuracy: 0.9908 - val_loss: 0.0214
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 264s 5s/step - accuracy: 0.9850 - loss: 0.0554 - val_accuracy: 0.9723 - val_loss: 0.0638
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 240s 4s/step - accuracy: 0.9770 - loss: 0.0697 - val_accuracy: 0.9908 - val_loss: 0.0322
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 273s 4s/step - accuracy: 0.9748 - loss: 0.0675 - val_accuracy: 0.9769 - v

In [ ]:
model.predict(img)

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_8" is incompatible with the layer: expected axis -1 of input shape to have value 186624, but received input with shape (1, 246016)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 256, 256, 3), dtype=float32)
  • training=False
  • mask=None

In [ ]:
# Predict the class
prediction = model.predict(img)

# Since it's binary classification, check if the output is greater than 0.5
predicted_class = 1 if prediction > 0.5 else 0

# Print the predicted class
print(f'Predicted class: {predicted_class}')


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_8" is incompatible with the layer: expected axis -1 of input shape to have value 186624, but received input with shape (1, 246016)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 256, 256, 3), dtype=float32)
  • training=False
  • mask=None

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img = image.load_img("/content/Data/test/COVID19/COVID19(468).jpg", target_size=(224, 224))  # Ensure size is (224, 224)
img = image.img_to_array(img)  # Convert image to numpy array
img = np.expand_dims(img, axis=0)  # Add batch dimension
img = img / 255.0  # Rescale to [0, 1]

# Predict the class
prediction = model.predict(img)

# For binary classification
predicted_class = 1 if prediction > 0.5 else 0
print(f'Predicted class: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted class: 0


In [ ]:
# Save the entire model to a file
model.save('my_model_Covid19.keras')


In [ ]:
from tensorflow.keras.models import load_model

# Load the model from file
model = load_model('my_model_Covid19.keras')

In [ ]:
import numpy as np

def predict(input_data):
    # Preprocess the input data (resize, scale, etc.)
    processed_data = preprocess(input_data)  # Implement this function based on your needs

    # Make predictions
    prediction = model.predict(processed_data)

    # Interpret the prediction
    predicted_class = np.argmax(prediction, axis=1)  # Adjust this based on your model's output
    return predicted_class


In [ ]:
img = "/content/Data/test/NORMAL/NORMAL(1306).jpg"
predict(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


array([0])

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess(input_data):
    # Load the image
    img = image.load_img(input_data, target_size=(224, 224))  # Adjust the target size based on your model
    # Convert the image to an array
    img_array = image.img_to_array(img)
    # Rescale the image
    img_array = img_array / 255.0
    # Expand dimensions to match the input shape of the model (batch size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict(input_data):
    # Preprocess the input data
    processed_data = preprocess(input_data)

    # Make predictions
    prediction = model.predict(processed_data)

    # Interpret the prediction
    predicted_class = np.argmax(prediction, axis=1)  # Use np.argmax for multi-class predictions
    return predicted_class

# Example usage
img_path = "/content/Data/test/NORMAL/NORMAL(1306).jpg"
result = predict(img_path)
print("Predicted class:", result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Predicted class: [0]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Define ImageDataGenerator for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    validation_split=0.2  # Split the data for validation
)

# Create training data generator
train_data = train_datagen.flow_from_directory(
    directory="/content/Data/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Set as training data
)

# Create validation data generator
validation_data = train_datagen.flow_from_directory(
    directory="/content/Data/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Set as validation data
)

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # For binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define a callback to save the best model
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model with the callback
model.fit(train_data, validation_data=validation_data,steps_per_epoch=8, epochs=10, callbacks=[checkpoint])


Found 1381 images belonging to 2 classes.
Found 345 images belonging to 2 classes.
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 62s 7s/step - accuracy: 0.6007 - loss: 3.5206 - val_accuracy: 0.7333 - val_loss: 0.6017
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 51s 7s/step - accuracy: 0.7316 - loss: 0.5385 - val_accuracy: 0.7333 - val_loss: 0.5348
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 71s 10s/step - accuracy: 0.7710 - loss: 0.4934 - val_accuracy: 0.7333 - val_loss: 0.4444
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 53s 7s/step - accuracy: 0.7429 - loss: 0.4278 - val_accuracy: 0.9362 - val_loss: 0.3281
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 46s 6s/step - accuracy: 0.9079 - loss: 0.3327 - val_accuracy: 0.9217 - val_loss: 0.2413
Epoch 6/10
4/8 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9095 - loss: 0.3373

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 30s 4s/step - accuracy: 0.9040 - loss: 0.3526 - val_accuracy: 0.9391 - val_loss: 0.2618
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 59s 7s/step - accuracy: 0.9039 - loss: 0.2724 - val_accuracy: 0.9420 - val_loss: 0.1877
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 48s 6s/step - accuracy: 0.8976 - loss: 0.2236 - val_accuracy: 0.9275 - val_loss: 0.2118
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 50s 7s/step - accuracy: 0.9096 - loss: 0.2806 - val_accuracy: 0.9304 - val_loss: 0.1795
Epoch 10/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 53s 7s/step - accuracy: 0.9527 - loss: 0.1793 - val_accuracy: 0.9507 - val_loss: 0.1379


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a callback to save the best model
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model with the callback
model.fit(train_data, validation_data , epochs=10, callbacks=[checkpoint])


NameError: name 'model' is not defined

## Malaria Detection

In [ ]:
!kaggle datasets download -d nipunarora8/malaria-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/nipunarora8/malaria-detection-dataset
License(s): copyright-authors
malaria-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -o /content/malaria-detection-dataset.zip

Archive:  /content/malaria-detection-dataset.zip
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_10.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_11.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_12.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_13.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_14.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_15.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_16.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_17.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_18.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_19.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense , Conv2D , MaxPool2D , Dropout , Flatten
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(directory="/content/Dataset/Train",target_size=(224,224),batch_size=32,class_mode='binary')

Found 416 images belonging to 2 classes.


In [ ]:
train_data.class_indices

{'Parasite': 0, 'Uninfected': 1}

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255, validation_split=0.2
)

test_data = test_datagen.flow_from_directory(directory="/content/Dataset/Test",target_size=(224,224),batch_size=32,class_mode='binary')

Found 134 images belonging to 2 classes.


In [ ]:
test_data.class_indices

{'Parasite': 0, 'Uninfected': 1}

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(256,256,3)))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=256, kernel_size=(3, 3), activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(rate=0.3))

model.add(Flatten())
model.add(Dense(units=64,activation="relu"))
model.add(Dropout(rate=0.50))
model.add(Dense(units=1,activation="sigmoid"))


model.compile(loss=keras.losses.binary_crossentropy,optimizer="adam",metrics=['acc'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 252, 252, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 126, 126, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 126, 126, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 124, 124, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       3,211,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,636,737 (13.87 MB)

 Trainable params: 3,636,737 (13.87 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from keras.regularizers import l2

model.add(Dense(units=64, activation="relu", kernel_regularizer=l2(0.01)))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint

model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the 2D outputs to 1D
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary output

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
# model.fit(train_data, steps_per_epoch=8, epochs=10, validation_data=test_data)

checkpoint = ModelCheckpoint('/content/drive/MyDrive/my_model_Maleria.keras',monitor='val_acc', save_best_only=True, mode='max')
model.fit(train_data, epochs=50, validation_data=test_data, callbacks=[checkpoint])


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/13 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.4882 - loss: 3.1944 - val_accuracy: 0.6716 - val_loss: 0.6606
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.5169 - loss: 0.6872 - val_accuracy: 0.6791 - val_loss: 0.6582
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.5877 - loss: 0.6527 - val_accuracy: 0.4478 - val_loss: 0.6941
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.6966 - loss: 0.5962 - val_accuracy: 0.4701 - val_loss: 0.7239
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.7314 - loss: 0.5255 - val_accuracy: 0.3209 - val_loss: 0.8451
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 51s 4s/step - accuracy: 0.5968 - loss: 0.6515 - val_accuracy: 0.6791 - val_loss: 0.6037
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.6524 - loss: 0.5512 - val_accuracy: 0.5970 - val_loss: 0.6466
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.7999 - loss: 0.5141 - val_accuracy: 0.5000 - val_loss: 0.7271
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8256 - loss: 0.4315 - val_accuracy: 0.3806 - val_loss: 0.9941
Epo

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img = image.load_img("/content/Dataset/Test/Uninfected/C3thin_original_IMG_20150608_162922_cell_205.png", target_size=(224, 224))  # Ensure size is (224, 224)
img = image.img_to_array(img)  # Convert image to numpy array
img = np.expand_dims(img, axis=0)  # Add batch dimension
img = img / 255.0  # Rescale to [0, 1]

# Predict the class
prediction = model.predict(img)

# For binary classification
predicted_class = 1 if prediction > 0.5 else 0
print(f'Predicted class: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted class: 1


In [ ]:
model.save('my_model_Maleria.keras')

In [ ]:
from google.colab import drive
drive.mount('/drive')

# Save the model to Google Drive
model.save('/drive/MyDrive/my_model_Maleria.keras')

Mounted at /drive


In [ ]:
from tensorflow.keras.models import load_model

# Load the model from file
model = load_model('my_model_Maleria.keras')

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess(input_data):
    # Load the image
    img = image.load_img(input_data, target_size=(224, 224))  # Adjust the target size based on your model
    # Convert the image to an array
    img_array = image.img_to_array(img)
    # Rescale the image
    img_array = img_array / 255.0
    # Expand dimensions to match the input shape of the model (batch size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict(input_data):
    # Preprocess the input data
    processed_data = preprocess(input_data)

    # Make predictions
    prediction = model.predict(processed_data)

    # Interpret the prediction
    predicted_class = 1 if prediction > 0.5 else 0
       # Use np.argmax for multi-class predictions
    return predicted_class

# Example usage
img_path = "/content/Dataset/Test/Uninfected/C3thin_original_IMG_20150608_163002_cell_159.png"
result = predict(img_path)
print("Predicted class:", result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted class: 1


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess(input_data):
    # Load the image
    img = image.load_img(input_data, target_size=(224, 224))  # Adjust the target size based on your model
    # Convert the image to an array
    img_array = image.img_to_array(img)
    # Rescale the image
    img_array = img_array / 255.0
    # Expand dimensions to match the input shape of the model (batch size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def predict(input_data):
    # Preprocess the input data
    processed_data = preprocess(input_data)

    # Make predictions
    prediction = model.predict(processed_data)

    # Interpret the prediction
    predicted_class = np.argmax(prediction, axis=1)  # Use np.argmax for multi-class predictions
    return predicted_class

# Example usage
img_path = "/content/Dataset/Test/Uninfected/C3thin_original_IMG_20150608_163002_cell_13.png"
result = predict(img_path)
print("Predicted class:", result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted class: [0]
